In [ ]:
import random
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
random.seed(time.time()) #To get a "really" random generator 

In [ ]:
class Cell (object) :
    
    next_ID = 0
    
    def __init__(self, state = "NORMAL", clone = 0, fitness = 1, mutation_rate = 0.001):
        self.state = state
        self.fitness = fitness
        self.clone = clone
        self.mutation_rate = mutation_rate
        self.ID = Cell.next_ID
        self.get_cure = False
        self.treatment_duration = 0
        Cell.next_ID += 1
        
    def __str__(self):
        return "Cell #{0} of state : {1} and clone type {2}".format(self.ID, self.state, self.clone)
    
    def mutate (self, new_clone_id):
        if self.state == "NORMAL":
            self.state = "CANCEROUS"
            
        self.clone = new_clone_id
        self.fitness = round(random.uniform(0,2), 3)
        self.mutation_rate = round(random.uniform(0, 0.05), 4)
        
    def treatment (self, alpha):
        self.get_cure == True
        self.fitness = self.fitness * alpha
        self.treatment_duration += 1

In [ ]:
class Tissue ( object ) :
    
    def __init__(self, omega, alpha, generation_time):
        self.generation_time = generation_time
        self.omega = omega
        self.alpha = alpha
        self.cancer_detection = False
        self.current_cure = None
        self.pop = []
        self.clones_pop = {0: {"fitness" : 1,"mu": 0.001, "freq" : 0}}
    
    def initial_population(self, N = 1000, prop_cancer = 0.01, init_clones=3):
        
        nb_cancer_cells = round(prop_cancer *N)
        
        for i in range(1,init_clones+1):
            self.clones_pop[i] = self.clones_pop.get(i, {"fitness" :round(random.uniform(0,2), 3), 
                                                 "mu": round(random.uniform(0, 0.05), 4), 
                                                "freq" : 0})
        
        for i in range(N):
            if (i < nb_cancer_cells) :
                state = "CANCEROUS"
                clone = random.choice(range(1, len(self.clones_pop.keys())))
                
            else :
                state = "NORMAL"
                clone = 0
            
            fitness = self.clones_pop[clone]["fitness"]
            mutation_rate = self.clones_pop[clone]["mu"]  
                
            new_cell = Cell(state, clone, fitness, mutation_rate)
            self.pop.append(new_cell)
            self.clones_pop[clone]["freq"] += 1 
            
    def stats(self):
        nb_cells = len(self.pop)
        nb_normal_cells = self.clones_pop[0]["freq"]
        nb_cancer_cells = nb_cells - nb_normal_cells
        nb_clones = 0
        
        average_fitness = sum([cell.fitness for cell in self.pop])/(nb_cells)
        
        for key in list(self.clones_pop.keys()):
            if (key != 0 and self.clones_pop[key]["freq"] > 0):
                nb_clones +=1
        
        if (nb_cancer_cells >= self.omega * nb_cells):
            self.cancer_detection = True
        else:
            self.cancer_detection = False
 
        return (nb_cells, nb_normal_cells, nb_cancer_cells, nb_clones, average_fitness)
            
    def targeted_treatment(self, n=5):
        
        if (n>len(self.clones_pop)) :
            n = len(self.clones_pop) -1
        
        # if the cancer has been detected and no cure in progress
        if (self.cancer_detection == True and self.current_cure == None):
            
            # find the biggest colonie of mutant clone to treat in prority
            self.current_cure = [clones[0] for clones in sorted({k:self.clones_pop[k] for k in self.clones_pop if k!=0}.items(), 
                                   key=lambda item: item[1]["freq"], 
                                   reverse=True)[:n]]
            
            for cell in self.pop:
                if (cell.clone in self.current_cure):
                    cell.treatment_duration = 0
                    cell.treatment(self.alpha)
                    
        # if there is already a cure on specific clone in progress, just continue it until the end of generation time                
        if(self.current_cure != None):
            remaining_cells_to_treat = 0
            for cell in self.pop:
                if(cell.clone in self.current_cure and cell.treatment_duration < self.generation_time):
                    cell.treatment(self.alpha)
                    remaining_cells_to_treat += 1
            
            if (remaining_cells_to_treat == 0):
                self.current_cure = None
                for cell in self.pop:
                    cell.get_cure == False
                    cell.treatment_duration = 0
                        
    
    def reproduce (self, weighted = True):
        
        if (weighted):
            candidat = random.choices(self.pop, weights=[c.fitness for c in self.pop], k=1)[0]
        else:
            candidat = random.choice(self.pop)

        new_cell = Cell(candidat.state, candidat.clone, candidat.fitness, candidat.mutation_rate)
        
        
        #Proba of mutation : give birth to a new clone type
        proba_mutation = random.uniform(0, 1) 
        if proba_mutation < new_cell.mutation_rate :
            new_clone_id=len(self.clones_pop.keys())
            new_cell.mutate(new_clone_id)
            self.clones_pop[new_cell.clone] = self.clones_pop.get(new_cell.clone, {"fitness" :new_cell.fitness, 
                                         "mu": new_cell.mutation_rate, 
                                        "freq" : 1})
        else:
            self.clones_pop[new_cell.clone]["freq"] += 1
        
        self.pop.append(new_cell)
    
    def get_apoptose(self, weighted = True):
        if (weighted):
            cell = random.choices(self.pop, weights=[1/(c.fitness+0.01) for c in self.pop], k=1)[0]
        else:
            cell = random.choice(self.pop)
            
        self.pop.remove(cell)
        self.clones_pop[cell.clone]["freq"] -= 1 

In [ ]:
def run (nb_runs=10000, 
         N=1000, 
         prop_cancer=0.1, 
         omega=0.2, 
         alpha=0.5, 
         T=20, 
         treatment = True, 
         nb_treatments = 5, 
         verbose = True,
         weighted_reproduction = True,
         weighted_apoptosis = True, 
         init_clones = 3):
    
    my_tissue = Tissue(omega, alpha, T)
    my_tissue.initial_population(N, prop_cancer, init_clones=init_clones)
    
    if verbose:
        print("Initilisation : ")
        print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells {}".format(my_tissue.stats()[0], my_tissue.stats()[1], my_tissue.stats()[2]) ) 
        print ( "\n" )
        
    nb_cells = []
    nb_normal_cells = []
    nb_cancer_cells = []
    nb_clones = []
    averages_fitness = []
    
    for i in range (nb_runs):
        #random.seed(time.time())
        my_tissue.reproduce(weighted_reproduction)
        my_tissue.get_apoptose(weighted_apoptosis)
        
        if (treatment) :
            my_tissue.targeted_treatment(n=nb_treatments)
        
        nb_cells.append(my_tissue.stats()[0])
        nb_normal_cells.append(my_tissue.stats()[1])
        nb_cancer_cells.append(my_tissue.stats()[2])
        nb_clones.append(my_tissue.stats()[3])        
        averages_fitness.append(my_tissue.stats()[4])
    
    df = pd.DataFrame({"Total cells" : nb_cells, 
                  "Normal cells" : nb_normal_cells, 
                  "Tumor cells" : nb_cancer_cells, 
                  "Number of clones": nb_clones, 
                  "Average fitness" : averages_fitness, 
                  })
        
    if verbose :
        print ( "Statistics after", nb_runs, "runs : ")
        print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells : {}".format(nb_cells[-1], nb_normal_cells[-1], nb_cancer_cells[-1], "\n" ) )
        print ( "\n" )
        
    return (df)

In [ ]:
#Without treatment :
N_runs = 10000
simulation1 = run(nb_runs=N_runs, 
                  N=1000, 
                  prop_cancer= 0.1, 
                  treatment=False, 
                  verbose=True,
                  weighted_reproduction = True,
                  weighted_apoptosis = True,
                  init_clones=10)

In [ ]:
#With treatment :
simulation2 = run (nb_runs=N_runs, 
                   N=1000, 
                   prop_cancer=0.1, 
                   omega=0.25, 
                   alpha=0.5, T=10, 
                   treatment=True, 
                   nb_treatments=2, 
                   verbose=True,
                   weighted_reproduction = True,
                   weighted_apoptosis = True,
                   init_clones=10)

In [ ]:
def make_plot(df1, df2, runs, fig_size=(22,14)):
    fig, axs = plt.subplots(2, 2, figsize=fig_size)

    axs[0, 0].plot(range(runs), df1["Normal cells"].values, label = "normal cells")
    axs[0, 0].plot(range(runs), df1["Tumor cells"].values, label = "cancerous cells")
    axs[0, 0].set_title('Simultation without treatment')
    axs[0, 0].legend()

    axs[0, 1].plot(range(runs), df2["Normal cells"].values, label = "normal cells")
    axs[0, 1].plot(range(runs), df2["Tumor cells"].values, label = "cancerous cells")
    axs[0, 1].set_title('Simultation with treatment')
    axs[0, 1].legend()

    axs[1, 0].plot(range(runs), df1["Number of clones"], label = "number of clones without treatment", color="lightcoral")
    axs[1, 0].plot(range(runs), df2["Number of clones"], label = "number of clones with treatment", color="forestgreen")
    axs[1, 0].set_title('Number of clones')
    axs[1, 0].legend()

    axs[1, 1].plot(range(runs), df1["Average fitness"].values, label = "average fitness without treatment", color="lightcoral")
    axs[1, 1].plot(range(runs), df2["Average fitness"].values, label = "average fitness with treatment", color="forestgreen")
    axs[1, 1].set_title('average fitness')
    axs[1, 1].legend()

    plt.show()

    return(axs)

In [ ]:
axs = make_plot(simulation1, simulation2, N_runs)